In [1]:
from IPython.core.display import HTML

# Resolvendo problemas por busca

## Agente de soluções de problemas

Agentes de soluções de problemas são agentes baseados em objetivos que buscam a melhor sequência de ações baseado nas opções de ações possíveis.

Problemas de malha aberta são problemas onde não é necessária uma nova percepção a cada passo em direção ao objetivo. Nesse tipo de problema, o agente tem um ambiente totalmente observável e a resolução do problema não lida com incertezas, sendo determinístico.

Problemas de malha fechada, por outro lado, apresentam incertezas no ambiente de forma que a melhor sequência pode variar

## Algoritmos de Busca

Há dois tipos de algortimos de busca, chamados de busca cega e busca informada.

- **Busca cega**: Não recebem nenhuma informação do quão próximos estão do objetivo;
- **Busca informada**: Recebem informações heurísticas da proximidade com o objetivo.

Para análisar a performance de algoritmos de busca, algumas características podem ser observadas, sendo elas:
    
- **Completude**: O algoritmo garante encontrar uma solução, se ela existir. E retorna um erro se a solução não existir.
- **Otimização**: O algoritmo garante encontrar a solução com menor custo.
- **Complexidade de tempo**: Quanto tempo o algoritmo leva para encontrar um solução.
- **Complexidade de espaço**: Quantidade de memória necessária para computar a solução.
- **Caminhos redundantes**: O grafo apresenta ciclos e o algoritmo cuida deles de alguma forma.

## Grafos

Antes de avançar para os algoritmos em si, falaremos um pouco sobre os grafos. Grafos são uma estrutura de dados que servem para modelar diversos tipos de problemas, desde máquinas de estados ou localizações. Os algoritmos de busca aqui citados, sejam eles de busca cega ou informada, utilizam essa estrutura.

A seguir uma breve explicação dos principais conceitos relacionados a grafos:

- **Node(nó)**: Um nó é uma entidade, algo que pode conter atributos e ser relacionado a outros nós. Um estado numa máquina de estados, uma cidade em um mapa, os animais em uma cadeia alimentar, dentre outros podem ser nós em um grafo.
- **Edge(aresta)**: Uma aresta é uma ligação entre nós de algum tipo. Uma rodovia entre duas cidades, por exemplo.
- **Direção da aresta**: Arestas podem ou não apresentar direções. Dados dois nós `A` e `B`. Se uma aresta for não direcionada, então é possível avançar nos dois sentidos, tanto de `A` para `B` quanto de `B` para `A`. Se uma aresta tiver direção, então só é possível ir de um nó para o outro, sem volta.
- **Peso de aresta**: Arestas também podem apresentar atributos(como o nome de uma rodovia, por exemplo), mas o mais relevante e conhecido é o peso. O peso de uma aresta pode reprentar uma distância ou dificuldade/facilidade para ir de um nó ao outro.

## Busca Cega

Algoritmos de busca cega são algoritmos cuja estratégia é visitar nó a nó até encontrar o objetivo. Alguns algoritmos deste tipo são:

- Breadth First Search (bfs)
- Depth First Search (dfs)
- Uniform Cost Search (ucs)

Para demonstrar a aplicação dos algoritmos, será usado de exemplo o "mapa" do primeiro `Super Mario Bros`. O jogo contém 32 estágios, separados em 8 mundos. O caminho normal é uma linha reta da primeira ao último estágio, mas as fases 1-2 e 4-2 apresentam atalhos(warps, como chamadados no jogo) para outros mundos.

Para a estrutura do grafo, foi utilizado um dicionário onde cada chave é um nó. E os valores são uma tupla cujo segundo elemento é a heurística, um valor que representa o quão longe se está da fase final, e o primeiro elemento é uma lista de tuplas com os vizinhos daquele nó e seus respectivos pesos. 

Obs.: O valor da heurística não é relevante para a busca cega, assim como todos os pesos estão com valor 0 para este grafo em especifico. Ambos os atributos estão aqui pois o grafo será reutilizado na busca informada.

O mapa do jogo está representado na célula abaixo, assim como algumas funções úteis:

In [2]:
from typing import List, Dict, Tuple
from queue import SimpleQueue

# Declara o tipo de dado Graph. É opcional uma vez que python não usa os type hints, mas é útil :)
Graph = Dict[str, Tuple[List[str], int]]

graph: Graph = {
    "1-1": ([("1-2", 0)], 31),
    "1-2": ([("1-3", 0), ("2-1", 0), ("3-1", 0), ("4-1", 0)], 30),
    "1-3": ([("1-4", 0)], 29),
    "1-4": ([("2-1", 0)], 28),
    "2-1": ([("2-2", 0)], 27),
    "2-2": ([("2-3", 0)], 26),
    "2-3": ([("2-4", 0)], 25),
    "2-4": ([("3-1", 0)], 24),
    "3-1": ([("3-2", 0)], 23),
    "3-2": ([("3-3", 0)], 22),
    "3-3": ([("3-4", 0)], 21),
    "3-4": ([("4-1", 0)], 20),
    "4-1": ([("4-2", 0)], 19),
    "4-2": ([("4-3", 0), ("5-1", 0), ("6-1", 0), ("7-1", 0), ("8-1", 0)], 18),
    "4-3": ([("4-4", 0)], 17),
    "4-4": ([("5-1", 0)], 16),
    "5-1": ([("5-2", 0)], 15),
    "5-2": ([("5-3", 0)], 14),
    "5-3": ([("5-4", 0)], 13),
    "5-4": ([("6-1", 0)], 12),
    "6-1": ([("6-2", 0)], 11),
    "6-2": ([("6-3", 0)], 10),
    "6-3": ([("6-4", 0)], 9),
    "6-4": ([("7-1", 0)], 8),
    "7-1": ([("7-2", 0)], 7),
    "7-2": ([("7-3", 0)], 6),
    "7-3": ([("7-4", 0)], 5),
    "7-4": ([("8-1", 0)], 4),
    "8-1": ([("8-2", 0)], 3),
    "8-2": ([("8-3", 0)], 2),
    "8-3": ([("8-4", 0)], 1),
    "8-4": ([], 0)
}

NEIGHBORS = 0
HEURISTIC = 1

NEIGHBOR = 0
WEIGHT = 1

def get_nodes(graph: Graph):
    """
    Retorna uma lista com todos os nós do grafo
    """
    return list(graph.keys())

def get_neighbors(graph: Graph, node: str):
    """
    Retorna uma lista com os nós vizinhos de um determinado nó, sem os pesos
    """
    return list(map(lambda n: n[NEIGHBOR], graph.get(node, ([], 0))[NEIGHBORS]))

def get_heuristic_value(graph: Graph, node: str):
    """
    Pega o valor da heurística de um determinado nó
    """
    return graph.get(node, ([], 0))[HEURISTIC]

def get_weight(graph: Graph, node_a: str, node_b: str):
    """
    Pega o peso atribuído de um nó A a um nó B
    """
    l = graph.get(node_a, ([], 0))[NEIGHBORS]
    for i in l:
        if i[NEIGHBOR] == node_b:
            return i[WEIGHT]

## BFS

O algoritmo **Breadth First Search**, ou BFS faz uma pesquisa por camadas, ou seja, ele visita todos os vizinhos antes de avançar para o próximo nível.
Este algoritmo sempre alcança o menor caminho possível (ótimo) se o grafo não tiver pesos na aresta.

In [3]:
HTML("""<img src="https://upload.wikimedia.org/wikipedia/commons/4/46/Animated_BFS.gif">""")

In [4]:
SELF = 0
PARENT = 1

def bfs(graph, initial_node, target_node):
    """
    Busca em largura. Pega como parâmetros o grafo, o nó inicial e o nó final(objetivo).
    Retorna a lista de visitados e o nó final, se acançavel
    """
    visited: List[Tuple[str, str]] = []
    fifo = SimpleQueue()
    
    #        (SELF, PARENT)
    fifo.put((initial_node, None))
    
    while not fifo.empty():
        current_node = fifo.get()
        
        visited.append(current_node)
        
        if current_node[SELF] == target_node:
            return visited, current_node[SELF]
        
        neighbors = get_neighbors(graph, current_node[SELF])
        for i in neighbors:
            if i not in list(map(lambda x: x[0], visited)):
                fifo.put((i, current_node[SELF]))
    
    return visited, None

In [5]:
def visited_list_to_dict(visited):
    """
    helper function para criar o caminho
    """
    return dict(visited)

def get_bfs_path(graph, initial_node, target_node):
    """
    processa o caminho a partir da resposta da função bfs
    """
    visited, node = bfs(graph, initial_node, target_node)
    if node == None:
        return []
    path = [visited[-1][0]]
    visited_d = visited_list_to_dict(visited)
    while True:
        if visited_d[path[-1]] is None:
            return path[::-1]
        path.append(visited_d[path[-1]])

In [6]:
# Como esperado, ele pega os warps para o mundo 4, e depois para o mundo 8, que é o menor caminho :)
get_bfs_path(graph, '1-1', '8-4')

['1-1', '1-2', '4-1', '4-2', '8-1', '8-2', '8-3', '8-4']

## Busca Informada

Algoritmos de busca informada utilizam de informações, ou dicas do quão próximos estão do objetivo. Desta forma, é possível empregar estratégias mais inteligentes para convergir ao objetivo.

Essas informações são chamadas de heurísticas, que por sua vez podem ser números exatos ou aproximações do quão próximo se está do objetivo. Vale ressaltar que estes são valores que em geral, ajudam a chegar no objetivo mais rapidamente, mas não será sempre o caso.

Exemplos de heurísticas:

- Distância entre dois nós, em um mapa por exemplo;
- Quantidade de peças em posição certa ou errada, no caso de quebra cabeças;

Alguns algoritmos para busca informada são:

- Greedy Search
- A* Search
- Graph Search

## A*

No exemplo prático deste tipo de algoritmos, reutilizaremos o mapa do super mario, porém com o algoritmo `A*`.
Este algoritmo utiliza como função de escolha para o próximo nó(`f(x)`), a soma acumulada até o nó atual(`g(x)`), mais o valor da heurística para aquele nó(`h(x)`).

f(x) = g(x) + h(x)

O algoritmo não é ótimo, uma vez que depende da heurística, que nem sempre terá os melhores valores. Mas sempre achará um caminho, desde que ele exista.

In [7]:
from queue import PriorityQueue

SELF_ = 0
PARENT_ = 1
G_VALUE = 2

def a_star(graph, start_node, target_node):
    """
    Recebe o grafo, o nó inicial e o nó objetivo.
    Retorna o caminho, se ele existir
    """
    
    # Fila de prioridade é útil para sempre pegar o valor mais baixo
    pfifo = PriorityQueue()
    #         (F_VALUE, (SELF_, PARENT_, G_VALUE))
    pfifo.put((0, (start_node, None, 0)))
    
    visited = {}
    
    while not pfifo.empty():
        cur_value, cur_node = pfifo.get()
        
        if cur_node[SELF] in visited:
            continue
        
        visited[cur_node[SELF]] = (cur_node[G_VALUE], cur_node[PARENT_])
        
        if cur_node[SELF] == target_node:
            return visited
        
        for neighbor in get_neighbors(graph, cur_node[SELF]):
            g = cur_node[G_VALUE] + get_weight(graph, cur_node[SELF], neighbor)
            f = g + get_heuristic_value(graph, neighbor)
            
            if visited.get(neighbor, False):
                if g < visited[neighbor][0]:
                    visited[neighbor] = (g, cur_node[SELF])
            else:
                pfifo.put((f, (neighbor, cur_node[SELF], g)))
    
    return None

In [11]:
def get_astar_path(graph, start_node, target_node):
    """
    Monta o caminho do A-estrela
    """
    result = a_star(graph, start_node, target_node)
    if result is None:
        print("No path")
    path = [target_node]
    while True:
        parent = result[path[-1]][1]
        if parent is None:
            break
        path.append(parent)
    return path[::-1]

In [9]:
# Esta é uma representação do grafo utilizado nos slides das aulas para este mesmo algoritmo
slide_graph = {
    "A": ([("B", 1), ("C", 2)], 5),
    "B": ([("D", 4), ("E", 6)], 3),
    "C": ([("F", 3), ("G", 2)], 4),
    "D": ([("E", 7)], 2),
    "E": ([], 6),
    "F": ([("H", 1)], 3),
    "G": ([("H", 2)], 1),
    "H": ([], 0)
}

In [12]:
# Mapa do Mario. Mesmo caminho do bfs, que é o caminho ótimo
get_astar_path(graph, '1-1', '8-4')

['1-1', '1-2', '4-1', '4-2', '8-1', '8-2', '8-3', '8-4']

In [13]:
get_astar_path(slide_graph, 'A', 'H')

['A', 'C', 'G', 'H']

In [14]:
get_astar_path(slide_graph, 'C', 'H')

['C', 'G', 'H']

## Procura em ambientes complexos

Os algoritmos apresentados até o momento se preocupam com o caminho para chegar de um nó a outro. Essa nem sempre é uma preocupação, e em alguns casos, apenas o estado final é importante. Para estes casos, temos os algoritmos de `busca local`.

Estes algoritmos tentam otimizar seus valores buscando valores melhores em seus vizinhos. Se o objetivo for uma elevação, então encontrar o maior pico é o processo de `hill climbing`. Se o objetivo é minimizar custos, então o processo é chamado `gradient descent`.

O algoritmo de hill climbing utiliza os seguintes passos [(ref*)](https://www.cs.iusb.edu/~danav/teach/c463/6_local_search.html):

- De um estado atual, se mova para algum dos estados vizinhos indo pra cima. Se houverem vários vizinhos com o valor maior, escolha o com o maior deles;
- Se o estado atual for o maior, o algoritmo encerra;
- Se chegar a um plateau (uma "planície"), pule para algum lado e continue a busca.

Algumas variações deste algoritmo podem ser implementadas, como:

- HC estocástico: ao invés do maior, escolha aleatoriamente o vizinho com valor maior do que o atual;
- HC primeira escolha: vá para o primeiro vizinho encontrado;
- Reinicio aleatório: Os movimentos para os lados são feitos de forma aleatória;
- HC evolutivo: representa soluções em potencial e vai mantendo os melhores até chegar a um resultado.

## Algoritmos Genéticos

Algoritmos genéticos são baseados na teoria da evolução de Darwin. "Indivíduos" ou estados aleatórios são gerados, e cada nova iteração, os melhores são mantidos e combinados entre si para gerar novos indivíduos com parâmetros melhores.

O exemplo prático neste caso é uma IA que aprende a jogar o jogo do dinossauro do google. O algoritmo foi retirado do blog [turing talks](https://medium.com/turing-talks/turing-talks-8-algoritmos-gen%C3%A9ticos-a791c25bd7ba). O código pode ser encontrado na pasta do portifólio no teams ou no repositório do git.

Para rodar o exemplo, é necessário a biblioteca do chrome_trex e o numpy. Para instalar usando o pip, segue o comando:

```
pip install --user git+https://github.com/GrupoTuringCodes/chrome-trex-rush@master numpy
```